# Data generation for segmentation model training

## Importing libraries

In [2]:
import cv2
import os
import shutil
import colorsys
import random
import numpy as np
import matplotlib.pyplot as plt
from collections import deque
from glob import glob
from openvino.runtime import Core
from tensorflow import reshape

2022-09-07 01:16:23.597476: I tensorflow/stream_executor/platform/default/dso_loader.cc:53] Successfully opened dynamic library libcudart.so.11.0


## Additional functions definition

The following functions were based on the [OAKarrange](https://github.com/cidimec/UCB-squirrels/blob/main/datasets_utils/OAKarrange.ipynb) notebook.

In [3]:
def isEmpty(scene):
    ''' Checks if in the current frame there is someone using the histogram of a blank laboratory '''
    hsv_scene = cv2.cvtColor(scene,cv2.COLOR_BGR2HSV)
    H, S, V = cv2.split(hsv_scene)
    H_array = H[S > 100].flatten()
    val, pos = np.histogram(H_array, 18)
    val[[0,3]] = 0

    return val.max()<1000

def create_dir(folder, force=True, verbose=False):
    '''   Create a directory if it doesn't exist  '''
    try:
        os.makedirs(folder)
        if verbose: print('Directory {} created succesfully.'.format(folder))   
    except:
        if force:
            if verbose: print('{} already exists. Creating a new one'.format(folder))
            shutil.rmtree(folder)
            os.makedirs(folder)
        else:
            if verbose: print('{} already exists.'.format(folder))
            pass



kernel = np.ones((5,5),np.uint8)

def bkgrd_Lab_substraction(frame, background, th=5, b_th=5):
    ''' Performs background substraction using the Lab color space, which helps to reduce light noise '''
    bkgrd_h = (cv2.cvtColor(background, cv2.COLOR_BGR2Lab))
    frgrd_h = (cv2.cvtColor(frame, cv2.COLOR_BGR2Lab))

    diff = cv2.subtract(bkgrd_h, frgrd_h)
    diff = cv2.cvtColor(diff, cv2.COLOR_BGR2GRAY)
    _, diff = cv2.threshold(diff, b_th, 255, cv2.THRESH_BINARY)

    diff = cv2.morphologyEx(diff, cv2.MORPH_CLOSE, kernel,1)
    diff = cv2.morphologyEx(diff, cv2.MORPH_DILATE, kernel,2)
    if diff.sum() / 1000000 > th:
        closed = fine_mask(diff)
        blured = cv2.GaussianBlur(closed,(3,3),0)
        closed = cv2.multiply(diff, closed)
        return True, diff, closed, blured
    else:
        return False, None, None, None
    
def fine_mask(mask):
    ''' Takes a raw mask as input and returns the biggest contour mask '''
    contours, _ = cv2.findContours(mask, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_NONE)
    c = max(contours, key = cv2.contourArea)
    out = np.zeros_like(mask)
    out = cv2.drawContours(out, [c], 0, 255, -1)
    return out


def random_colors(N, bright=True):
    ''' Generate random colors  '''
    brightness = 1.0 if bright else 0.7
    hsv = [(i / N, 1, brightness) for i in range(N)]
    colors = list(map(lambda c: colorsys.hsv_to_rgb(*c), hsv))
    # random.shuffle(colors)
    return colors

color = random_colors(1)[0]

def apply_mask(frame, mask, color, alpha=0.5):
    ''' Apply the given mask to the image '''
    image = frame.copy()
    for c in range(3):
        image[:, :, c] = np.where(mask == 1,
                                  image[:, :, c] *
                                  (1 - alpha) + alpha * color[c] * 255,
                                  image[:, :, c])
    return image

## Background substraction

In [5]:
images_dir = '/home/christian/Documents/Datasets/OakGait16/frames/'
clips_directory = '/home/christian/Documents/Datasets/OakGait16/clips'
frs_dir = 'rgb_segmentation/'
mks_dir = 'masks/'
verbose = True
#subjects = sorted(os.listdir(os.path.join(clips_directory, '105')))
subjects = ['000', '001', '002', '003', '004', '005', '006', '007', '008', '009', '010', '011', '012', '013', '014', '015', '016']
nclips = {'nm':5, 'bg':5, 'cl':5}

print(subjects)

['000', '001', '002', '003', '004', '005', '006', '007', '008', '009', '010', '011', '012', '013', '014', '015', '016']


In [6]:
views = ['060', '075', '090', '105', '120']
for view in views:
    frames_dir = os.path.join(images_dir, view, frs_dir)
    masks_dir = os.path.join(images_dir, view, mks_dir)
    print(f'PROCESSING VIEW: {view}')

    for subject in subjects:

        print(f'Processing subject: {subject} view: {view}....                                  ')
        create_dir(os.path.join(frames_dir, subject), force=True)
        create_dir(os.path.join(masks_dir, subject), force=True)

        subject_dir = os.path.join(clips_directory, view, subject)
        walks = os.listdir(subject_dir)
        
        for j, walk in enumerate(['nm', 'bg', 'cl']):


            walk_dir = os.path.join(subject_dir, walk)
            background = cv2.imread(os.path.join(walk_dir, f'background.png'))
            clips = sorted(os.listdir(walk_dir))
            
            save_back_path = os.path.join(frames_dir, subject, f'{walk}-background.png')
            cv2.imwrite(save_back_path, background)
            
            for i, clip in enumerate(clips):
                if clip[-4:] == '.avi' and nclips[walk]>=i:
                    clip_path = os.path.join(walk_dir, clip)
                    sub_dir = os.path.join(frames_dir, subject, walk, clip.split('.')[0])
                    mask_dir = os.path.join(masks_dir, subject, walk, clip.split('.')[0])
                    create_dir(sub_dir, force=True)
                    create_dir(mask_dir, force=True)
                    
                    cap = cv2.VideoCapture(clip_path)
                    cnt = 0
                    while True:
                        ret, frame = cap.read()
                        if ret:
                            try:
                                if subject in ['000', '008', '009', '010', '011', '012', '013', '014', '015', '016']:
                                    ok, diff_gray, closed, blured = bkgrd_Lab_substraction(frame, background, b_th=10)
                                else:
                                    ok, diff_gray, closed, blured = bkgrd_Lab_substraction(frame, background)
                                if ok:
                                    
                                    frame_path = f'{sub_dir}/{str(cnt).zfill(4)}.jpg'
                                    mask_path = f'{mask_dir}/{str(cnt).zfill(4)}.jpg'
                                    frame = cv2.resize(frame, (960,540), interpolation=cv2.INTER_AREA)
                                    closed = cv2.resize(closed, (960,540), interpolation=cv2.INTER_AREA)

                                    if np.sum(closed[540//2+40:,:]) > 3500000 and np.sum(closed[:540//2+40,:]) > 2000000:
                                        cnt +=1
                                        cv2.imwrite(frame_path, frame)
                                        cv2.imwrite(mask_path, closed)
                                        
                                    if verbose:
                                        masked = apply_mask(frame.copy(), closed.astype('bool'), color)

                            except:
                                pass
                        else:
                            break
                        if cv2.waitKey(1) == ord('q'):
                            break
                    cap.release()
    print('', end="\r")
cv2.destroyAllWindows()

PROCESSING VIEW: 060
Processing subject: 000 view: 060....                                  
Processing subject: 001 view: 060....                                  
Processing subject: 002 view: 060....                                  
Processing subject: 003 view: 060....                                  
Processing subject: 004 view: 060....                                  
Processing subject: 005 view: 060....                                  
Processing subject: 006 view: 060....                                  
Processing subject: 007 view: 060....                                  
Processing subject: 008 view: 060....                                  
Processing subject: 009 view: 060....                                  
Processing subject: 010 view: 060....                                  
Processing subject: 011 view: 060....                                  
Processing subject: 012 view: 060....                                  
Processing subject: 013 view: 060....      

## Region of interest & Ground truth obtention

In [7]:
ie = Core()
device = "CPU"
model = ie.read_model(model="/home/christian/Documents/CNN-Gait-Recognition-Offline/models/public/mobilenet-ssd/FP32/mobilenet-ssd.xml")
compiled_model = ie.compile_model(model=model, device_name=device)
input_layer_ir = next(iter(compiled_model.inputs))
output_layer_ir = next(iter(compiled_model.outputs))

def find_ROI(img):
        img = cv2.cvtColor(img, cv2.COLOR_RGB2BGR)
        input_image = cv2.dnn.blobFromImage(img, size=(300, 300), ddepth=cv2.CV_8U)

        input_image = reshape(input_image, [-1,3,300,300])
        result = compiled_model.infer_new_request({0: input_image})

        x_min = next(iter(result.values()))[:,:,np.where(next(iter(result.values()))[0,0,:,1] == 15.),3]*300
        y_min = next(iter(result.values()))[:,:,np.where(next(iter(result.values()))[0,0,:,1] == 15.),4]*300
        x_max = next(iter(result.values()))[:,:,np.where(next(iter(result.values()))[0,0,:,1] == 15.),5]*300
        y_max = next(iter(result.values()))[:,:,np.where(next(iter(result.values()))[0,0,:,1] == 15.),6]*300

        (x, y, w, h) = int(x_min), int(y_min), int(x_max), int(y_max)

        offsets = [10, 10, 10, 10]
        x = x - offsets[0] if x - offsets[0] >=0 else 0
        y = y - offsets[3] if y - offsets[3] >=0 else 0
        w = w + offsets[1]+offsets[0] if w + offsets[1]+offsets[0] <=300 else 300
        h = h + offsets[3]+offsets[1] if h + offsets[3]+offsets[1] <=300 else 300

        
        return x,y,w,h

In [8]:
images_dir = '/home/christian/Documents/Datasets/OakGait16/frames/'
rois_dir = '/home/christian/Documents/Datasets/OakGait16/rois/'
gt_rois_dir = '/home/christian/Documents/Datasets/OakGait16/gt_rois/'
frs_dir = 'rgb_segmentation/'
mks_dir = 'masks/'
views = ['060','075','090','105','120']
training_silhouettes = '/home/christian/Documents/Datasets/OakGait16/rgb_silhouettes/'
masked_silhouettes = '/home/christian/Documents/Datasets/OakGait16/masked_silhouettes/'

In [9]:
create_dir(training_silhouettes, force=True)
create_dir(masked_silhouettes, force=True)
c = 0
for view in views:
    frames_dir = os.path.join(images_dir, view, frs_dir)
    masks_dir = os.path.join(images_dir, view, mks_dir)
    rep_dir = os.path.join(rois_dir, view)
    gt_dir = os.path.join(gt_rois_dir, view)
    print(f'GENERATING GAIT REPRESENTATIONS FROM VIEW: {view}')
    subjects = sorted(os.listdir(os.path.join(images_dir, view, frs_dir)))
    
    for subject in subjects:
#     for subject in ['022', '023', '025']:
        print(f'Processing subject: {subject} view: {view}')
        

        for j, walk in enumerate(['nm','bg','cl']):
#         for j, walk in enumerate(['nm']):
            seq_dir = os.path.join(frames_dir, subject, walk)
            seqs = sorted(os.listdir(seq_dir))
            
            print(f'Processing: {seqs}')
            for s, seq in enumerate(seqs):

                seq_frames_dir = os.path.join(seq_dir, seq)
                lfiles = sorted(os.listdir(seq_frames_dir))
                cnt = 0
                for idimg, path in enumerate(lfiles):
                    
                    
                        img = cv2.imread(os.path.join(seq_frames_dir, path))
                        img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
                        img = cv2.resize(img,(300,300))
                        mask = cv2.imread(os.path.join(masks_dir, subject, walk, seq, path),0)
                        mask = cv2.resize(mask,(300,300))


                        try: 
                            x,y,w,h = find_ROI(img)
                            bbox =  [x, y, w, h]
                            gti = mask[bbox[1]:bbox[3], bbox[0]:bbox[2]]
                            roi = cv2.cvtColor(img[bbox[1]:bbox[3], bbox[0]:bbox[2]], cv2.COLOR_BGR2RGB)
                            
                            gti_path = os.path.join(masked_silhouettes, f'{subject}-{seq}-{str(c).zfill(6)}.jpg')
                            roi_path = os.path.join(training_silhouettes, f'{subject}-{seq}-{str(c).zfill(6)}.jpg')

                            if roi.shape[1] > 60  or roi.shape[1] < 150:
                                cv2.imwrite(gti_path, cv2.resize(gti, (128,128), interpolation=cv2.INTER_AREA))
                                cv2.imwrite(roi_path, cv2.resize(roi, (128,128), interpolation=cv2.INTER_AREA))
                                c += 1
                            
                        except:
                            pass           

GENERATING GAIT REPRESENTATIONS FROM VIEW: 060
Processing subject: 000 view: 060
Processing: ['nm-01', 'nm-02', 'nm-03', 'nm-04', 'nm-05']


2022-09-07 02:38:44.075151: I tensorflow/stream_executor/platform/default/dso_loader.cc:53] Successfully opened dynamic library libcuda.so.1
2022-09-07 02:38:45.348495: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:937] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-09-07 02:38:45.509304: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1733] Found device 0 with properties: 
pciBusID: 0000:01:00.0 name: NVIDIA GeForce 940MX computeCapability: 5.0
coreClock: 1.2415GHz coreCount: 3 deviceMemorySize: 3.95GiB deviceMemoryBandwidth: 14.30GiB/s
2022-09-07 02:38:45.546486: I tensorflow/stream_executor/platform/default/dso_loader.cc:53] Successfully opened dynamic library libcudart.so.11.0
2022-09-07 02:38:46.434488: I tensorflow/stream_executor/platform/default/dso_loader.cc:53] Successfully opened dynamic library libcublas.so.11
2022-09-07 02:38:46.434644: I tensorflow/stream_executor/platfor

Processing: ['bg-01', 'bg-02', 'bg-03', 'bg-04', 'bg-05']
Processing: ['cl-01', 'cl-02', 'cl-03', 'cl-04', 'cl-05']
Processing subject: 001 view: 060
Processing: ['nm-01', 'nm-02', 'nm-03', 'nm-04', 'nm-05']
Processing: ['bg-01', 'bg-02', 'bg-03', 'bg-04', 'bg-05']
Processing: ['cl-01', 'cl-02', 'cl-03', 'cl-04', 'cl-05']
Processing subject: 002 view: 060
Processing: ['nm-01', 'nm-02', 'nm-03', 'nm-04', 'nm-05']
Processing: ['bg-01', 'bg-02', 'bg-03', 'bg-04', 'bg-05']
Processing: ['cl-01', 'cl-02', 'cl-03', 'cl-04', 'cl-05']
Processing subject: 003 view: 060
Processing: ['nm-01', 'nm-02', 'nm-03', 'nm-04', 'nm-05']
Processing: ['bg-01', 'bg-02', 'bg-03', 'bg-04', 'bg-05']
Processing: ['cl-01', 'cl-02', 'cl-03', 'cl-04', 'cl-05']
Processing subject: 004 view: 060
Processing: ['nm-01', 'nm-02', 'nm-03', 'nm-04', 'nm-05']
Processing: ['bg-01', 'bg-02', 'bg-03', 'bg-04', 'bg-05']
Processing: ['cl-01', 'cl-02', 'cl-03', 'cl-04', 'cl-05']
Processing subject: 005 view: 060
Processing: ['nm-0